<a href="https://colab.research.google.com/github/edinaldoab/challenge_data_science_2/blob/main/notebooks/semana02_tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 2
### Contextualização


Aqui será empregado uma abordagem de **Ciências de Dados**! Toda a base tratada e disponibilizada no formato PARQUET será explorada e tratada para finalmente elencar e criar os modelos de Machine Learning.
<br>
<br>
**Recordando: Na primeira semana do desafio, todos os esforços foram dedicados ao tratamento da base de dados bruta disponibilizada pelos desafiadores.**

# 1. Preparação do Ambiente

## a. Instalação das dependências

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

## b. Inicialização da SparkSession

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("challenge_imob") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## c. Montagem do Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## d. Inicializando a  UI do Spark 

In [ ]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dados/authTokenngrok.txt', 'r') as file:
  content = file.read()
  file.close()

my_authtoken = str(content)

In [ ]:
get_ipython().system_raw('./ngrok authtoken ' + my_authtoken)
get_ipython().system_raw('./ngrok http 4050 &')

!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://331a-34-75-20-66.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://331a-34-75-20-66.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
spark

Carregamento dos dados:

# 2. Carregamento e tratamento dos dados

## a. Leitura do dataset

In [6]:
path='/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dataset_parquet'

data = spark.read.parquet(path)

In [7]:
data.show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+---------------+----------+----------+-----+-----+-------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|         bairro|      zona|condominio| iptu| tipo|  valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+---------------+----------+----------+-----+-----+-------+
|00790b85-56e3-43a...|   11|       166|      166|        5|[Academia, Churra...|      4|     4|       Usado| Apartamento|Residencial|   2|    Jacarepaguá|Zona Oeste|      2100| 4600|Venda|1750000|
|007f8099-8e1d-45f...|    4|       640|      640|        5|[Animais permitid...|     11|     3|       Usado| Apartamento|Residencial|null|           Urca|  Zona Sul|      3060|20030|Venda|3800000|
|00b23c6d-0e9d-

## b. Seleção de features

Aqui devem ser removidas as colunas que possuem apenas um valor atribuído, de acordo com o que foi elaborado na primeira semana. Esta etapa é muito importante para se reduzir a complexidade dos dados e aumentar a velocidade de ajuste do modelo.

In [9]:
data\
    .select('tipo')\
    .distinct()\
    .show(5)

+-----+
| tipo|
+-----+
|Venda|
+-----+



In [10]:
data\
    .select('tipo_anuncio')\
    .distinct()\
    .show(5)

+------------+
|tipo_anuncio|
+------------+
|       Usado|
+------------+



In [11]:
data\
    .select('tipo_unidade')\
    .distinct()\
    .show(5)

+------------+
|tipo_unidade|
+------------+
| Apartamento|
+------------+



In [12]:
data\
    .select('tipo_uso')\
    .distinct()\
    .show(5)

+-----------+
|   tipo_uso|
+-----------+
|Residencial|
+-----------+



As colunas `area_total` e `area_util` têm um potencial maior de aderência ao modelo. Assim, analisa-se:

In [15]:
data\
    .select('area_total','area_util')\
    .show(5)

+----------+---------+
|area_total|area_util|
+----------+---------+
|       166|      166|
|       640|      640|
|        50|       50|
|       160|      160|
|        52|       52|
+----------+---------+
only showing top 5 rows



In [21]:
data\
    .select('area_util', 'area_total')\
    .na\
    .drop()\
    .show(5)

+---------+----------+
|area_util|area_total|
+---------+----------+
|      166|       166|
|      640|       640|
|       50|        50|
|      160|       160|
|       52|        52|
+---------+----------+
only showing top 5 rows



In [17]:
from pyspark.sql import functions as f

dados_without_na = data\
    .select('area_util', 'area_total')\
    .na\
    .drop()

quantidade_total = dados_without_na.count()
quantidade_iguais = dados_without_na.filter(f.col('area_util') == f.col('area_total')).count()
quantidade_distintas = quantidade_total - quantidade_iguais

print(f'Quantidade de linhas: {quantidade_total}')
print(f'Quantidade de linhas com area_util == area_total: {quantidade_iguais}')
print(f'Quantidade de linhas diferentes: {quantidade_distintas}')

Quantidade de linhas: 57368
Quantidade de linhas com area_util == area_total: 55384
Quantidade de linhas diferentes: 1984


Chega-se a algumas conclusões:
<br>
<br>
*  das 57368 linhas, 55384 possuem valores iguais nas colunas `area_total` e `area_util`
*  pode-se escolher somente uma, uma vez que a maioria das informações são repetidas

Para esta escolha, levou-se em conta a coluna com a menor quantidade de dados nulos:

In [29]:
colunas = ['area_total', 'area_util']

data\
    .select([f.count(f.when(data[c].isNull(), True)).alias(c) for c in colunas])\
    .show()

+----------+---------+
|area_total|area_util|
+----------+---------+
|      9194|        0|
+----------+---------+



In [30]:
'''
cont = []
for c in colunas:
    cont.append(f.count(f.when(data[c].isNull(), True)).alias(c))

data.select(cont).show()

+----------+---------+
|area_total|area_util|
+----------+---------+
|      9194|        0|
+----------+---------+



A `area_total` não possui nenhum valor nulo! Pode-se então eliminar a coluna `area_util`.

In [31]:
data = data\
    .drop('area_total','tipo_anuncio','tipo_unidade','tipo_uso','tipo')

In [32]:
data.show(5)

+--------------------+-----+---------+---------+--------------------+-------+------+----+---------------+----------+----------+-----+-------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|         bairro|      zona|condominio| iptu|  valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+---------------+----------+----------+-----+-------+
|00790b85-56e3-43a...|   11|      166|        5|[Academia, Churra...|      4|     4|   2|    Jacarepaguá|Zona Oeste|      2100| 4600|1750000|
|007f8099-8e1d-45f...|    4|      640|        5|[Animais permitid...|     11|     3|null|           Urca|  Zona Sul|      3060|20030|3800000|
|00b23c6d-0e9d-4be...|    1|       50|        1|[Academia, Churra...|      2|     0|   1|Parada de Lucas|Zona Norte|       363|   97| 192000|
|012aebd7-278f-4b4...|    0|      160|        4|          [Elevador]|      3|     1|   3|        Ipanema|  Zona Sul|      1530| 7440|3490000|
|0267d

## c. Conversão dos tipos das colunas

In [33]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



Os dados numéricos estão tipados em sua maioria como string, enquanto deveriam receber as seguintes conversões:



In [40]:
from pyspark.sql.types import IntegerType, DoubleType

In [41]:
data = data\
    .withColumn('andar', data['andar'].cast(IntegerType()))\
    .withColumn('area_util', data['area_util'].cast(DoubleType()))\
    .withColumn('banheiros', data['banheiros'].cast(IntegerType()))\
    .withColumn('quartos', data['quartos'].cast(IntegerType()))\
    .withColumn('suites', data['suites'].cast(IntegerType()))\
    .withColumn('condominio', data['condominio'].cast(DoubleType()))\
    .withColumn('vaga', data['vaga'].cast(IntegerType()))\
    .withColumn('iptu', data['iptu'].cast(DoubleType()))\
    .withColumn('valor', data['valor'].cast(DoubleType()))

In [42]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



## d. Tratamento da coluna `caracteristicas`

In [45]:
data\
    .select('caracteristicas')\
    .show(100, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+
|caracteristicas                                                                                                                                   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|[Academia, Churrasqueira, Piscina, Playground, Salão de festas, Elevador, Condomínio fechado, Portão eletrônico, Portaria 24h, Animais permitidos]|
|[Animais permitidos, Portão eletrônico]                                                                                                           |
|[Academia, Churrasqueira, Piscina, Playground, Salão de festas, Condomínio fechado, Portaria 24h, Animais permitidos]                             |
|[Elevador]                                                                                               

A consulta da coluna característica mostra que algumas listas atribuídas estão vazias. A tratativa para este caso, é convertê-las para `null`:

In [46]:
data = data.withColumn('caracteristicas', f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col('caracteristicas')))

In [51]:
data.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+----------+----------+-------+---------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|      zona|condominio|   iptu|    valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+----------+----------+-------+---------+
|00790b85-56e3-43a...|   11|    166.0|        5|[Academia, Churra...|      4|     4|   2|         Jacarepaguá|Zona Oeste|    2100.0| 4600.0|1750000.0|
|007f8099-8e1d-45f...|    4|    640.0|        5|[Animais permitid...|     11|     3|null|                Urca|  Zona Sul|    3060.0|20030.0|3800000.0|
|00b23c6d-0e9d-4be...|    1|     50.0|        1|[Academia, Churra...|      2|     0|   1|     Parada de Lucas|Zona Norte|     363.0|   97.0| 192000.0|
|012aebd7-278f-4b4...|    0|    160.0|        4|          [Elevador]|      3|     1|   3|     